In [42]:
import pandas as pd
import praw

In [43]:
qbs_and_clusters = pd.read_csv("https://raw.githubusercontent.com/jake-lukasik/NFL-Player-Clustering/refs/heads/main/filtered-data/QBS-and-Clusters.csv").drop(columns=["Unnamed: 0"])
display(qbs_and_clusters)

,Player,Team,Cluster
0,Joe Burrow,CIN,2
1,Jared Goff,DET,2
2,Baker Mayfield,TAM,2
3,Geno Smith,SEA,0
4,Sam Darnold,MIN,2
5,Lamar Jackson,BAL,2
6,Patrick Mahomes,KAN,0
7,Aaron Rodgers,NYJ,0
8,Justin Herbert,LAC,0
9,Brock Purdy,SFO,3


In [44]:
# define a function that takes in QB name and num of comments you
# want to fetch: using these reddit comments, we can perform simple 
# sentiment analysis on each quarterback
def get_reddit_posts(player_name, max_results=25):
    # set up the reddit client
    reddit_client = praw.Reddit(
        client_id="7P0ZI6dh924lFvasX-jE-A",
        client_secret="AIuVhL1m4LGLTi-ZYQPyslfeGUlLaQ",
        user_agent="nfl-scraper"
    )

    # init subreddit and matched comments we want to look at
    subreddit = reddit_client.subreddit("nfl")
    matched_comments = []

    # loop through results of searching the QB name and find comments discussing them
    for submission in subreddit.search(query=player_name, sort="relevance", limit=50):
        try:
            submission.comments.replace_more(limit=0)
            for comment in submission.comments.list():
                if player_name.lower() in comment.body.lower():
                    matched_comments.append(comment.body)
                    if len(matched_comments) >= max_results:
                        return matched_comments
        except Exception as e:
            print(f"Error reading submission {submission.id}: {e}")
            continue

    return matched_comments

In [45]:
qbs_and_clusters["reddit_comments"] = None

for idx, row in qbs_and_clusters.iterrows():
    player_name = row["Player"]
    comments = get_reddit_posts(player_name)
    qbs_and_clusters.at[idx, "reddit_comments"] = comments
display(qbs_and_clusters)

,Player,Team,Cluster,reddit_comments
0,Joe Burrow,CIN,2,[Joe Burrow bought those so they could attack ...
1,Jared Goff,DET,2,"[Jared Goff is elite at play action, Jared Gof..."
2,Baker Mayfield,TAM,2,"[If you don't like Baker Mayfield, fuck you., ..."
3,Geno Smith,SEA,0,"[GENO SMITH BENDS THE KNEE!!!, Geno Smith and ..."
4,Sam Darnold,MIN,2,[Surely Sam Darnold succeeding Geno Smith will...
5,Lamar Jackson,BAL,2,[I feel so fucking lucky to be a Ravens and La...
6,Patrick Mahomes,KAN,0,"[Super Bowl Mvp Patrick Mahomes, ""Chiefs QB Pa..."
7,Aaron Rodgers,NYJ,0,[It must be weird to be spoken to with your fu...
8,Justin Herbert,LAC,0,"[Maybe I’m Justin Herbert 🤔, This just in…Just..."
9,Brock Purdy,SFO,3,[Yeah really depends on how you view it lol I ...


In [46]:
# iterate over 0-3 and isolate each cluster into their own df
cluster_dfs = [
    qbs_and_clusters[qbs_and_clusters["Cluster"] == i]
    for i in range(4)
]
display(cluster_dfs[0]) # access each individual cluster like this

,Player,Team,Cluster,reddit_comments
3,Geno Smith,SEA,0,"[GENO SMITH BENDS THE KNEE!!!, Geno Smith and ..."
6,Patrick Mahomes,KAN,0,"[Super Bowl Mvp Patrick Mahomes, ""Chiefs QB Pa..."
7,Aaron Rodgers,NYJ,0,[It must be weird to be spoken to with your fu...
8,Justin Herbert,LAC,0,"[Maybe I’m Justin Herbert 🤔, This just in…Just..."
10,Kyler Murray,ARI,0,"[* **No. 1 overall — Oklahoma’s Kyler Murray, ..."
11,Bo Nix,DEN,0,[that went about 66 air yards and i thought bo...
12,Matthew Stafford,LAR,0,"[That’s super bowl champion Matthew Stafford, ..."
14,C.J. Stroud,HOU,0,[I've seen people claim that it's been disprov...
15,Jayden Daniels,WAS,0,[Does Jayden Daniels not care about winning? m...
16,Caleb Williams,CHI,0,"[Caleb Williams had plenty of flaws, although ..."


In [ ]:
#qbs_and_clusters.to_csv("C:/Users/jakel/OneDrive/Desktop/GitHub/NFL-Player-Clustering/filtered-data/QBS-and-Comments.csv")